In [ ]:
# Migrate data to S3 and use that instead of doing locally
# https://www.sqlshack.com/getting-started-with-amazon-s3-and-python/

In [102]:
from os import listdir
from pandas import concat, read_csv
from datetime import datetime as dt
# from geopandas import GeoDataFrame, points_from_xy

In [76]:
# Still need to put the MODIS data in a better location. Current path is not ideal.
path = '../../../Fires/MODIS Archive'
ext = '.csv'

In [89]:
def timer(func):
    def wrapper(*args, **kwargs):
        start = dt.now()
        print('Started: ', start)

        result = func(*args, **kwargs)

        end = dt.now()
        print('Ended: ', end)
        print('Total: ', end - start)
        
        return result

    return wrapper

In [99]:
@timer
def combine_modis(path, confidence_level=80):
    """
    This function creates a single csv from annual MODIS fire alerts downloaded from
    FIRMS (https://firms.modaps.eosdis.nasa.gov/download/). The input is a file path
    to the directory that contains the individual csv files and a confidence level
    threshold (default = 80). The function will concatenate rows where the confidence
    level is at or above the confidence level threshold. For example:

    combine_modis(path='./mydir', confidence_level=80)

    Returns a single csv with rows containing a confidence level greater than or
    equal to 80.
    """
    ext = '.csv'

    files = [path + '/' + f for f in listdir(path) if f.endswith(ext)]

    l = []

    print('Reading files...')
    for file in files:
        print('-Reading ', file[29:])  # Find a better way to print file name
        df = read_csv(file)
        df = df.loc[df['confidence'] >= confidence_level]
        l.append(df)

    out = concat(l)
    
    print('Done.')

    return out


In [80]:
# # This one might actually be a little slower. Need to do more tests.
# @timer
# def combine_modis_alt(path, confidence_level=80):
#     """
#     This function creates a single csv from annual MODIS fire alerts downloaded from
#     FIRMS (https://firms.modaps.eosdis.nasa.gov/download/). The input is a file path
#     to the directory that contains the individual csv files and a confidence level
#     threshold (default = 80). The function will concatenate rows where the confidence
#     level is at or above the confidence level threshold. For example:

#     combine_modis(path='./mydir', confidence_level=80)

#     Returns a single csv with rows containing a confidence level greater than or
#     equal to 80.
#     """
#     ext = '.csv'

#     files = [path + '/' + f for f in listdir(path) if f.endswith(ext)]

#     out = []

#     for file in files:
#         df = read_csv(file)
#         df = df.loc[df['confidence'] >= confidence_level]
#         out.append(df)

#     return concat(out)


In [28]:
# files = [path + '/' + f for f in listdir(path) if f.endswith(ext)]
# files

In [ ]:
# combined = concat([read_csv(f) for f in files])
# final = combined.loc[combined['confidence']>=80]

In [91]:
combined = combine_modis(path=path, confidence_level=80)

Started:  2021-02-22 15:08:27.467307
Ended:  2021-02-22 15:13:08.178320
Total:  0:04:40.711013


In [92]:
combined.shape

(26201510, 15)

In [94]:
combined.to_csv('modis_alerts_2001_2020.csv')

In [119]:
# Doesn't work right now...
# gdf = GeoDataFrame(
#     combined,
#     geometry=points_from_xy(
#         df.longitude,
#         df.latitude
#     )
# )